## Before running this, make sure mk2-sat-hat-3 exists in your host file, and the port forward has been created (ssh -L 13450:192.168.6.2:13450 -L 2222:192.168.42.25:22 pi@mk2-sat-hat-3).

In [ ]:
from pysi.leops.session import new_session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
from eyeq import Client, Block, block
from eyeq.client import samples
client = Client("tcp://localhost:13450")

In [ ]:
os.environ['JUPYTERHUB_USER'] = os.environ['USER']
session = new_session('mk2-sat-hat-3:6020', 0)

In [ ]:
# EyeQ setup.
raw_stream = client.create_stream(name="raw_stream", layers=[
    samples.StoreReaderStream(name='SAMPLES', start_block=0, end_block=15000)
])

In [ ]:
%si node 22
%si set desired_state 3 safemode_override 1 sample_to 1
%si sleep 1
%si set processor_enabled 1 gndwdt 86400

In [ ]:
%si ping 24
%si ping 25

In [ ]:
%si node 22
%si get radio*

In [ ]:
def get_sample_offset(store="SAMPLES"):
    return [x for x in client.list_stores().stores if x.name == "SAMPLES"][0].block_offset

In [ ]:
get_sample_offset()

In [ ]:
%si node 22
%si get sample_for

In [ ]:
start_offset = get_sample_offset()
# Test sampling
%si node 22
%si set sample_for 20 radio_rx_gain 30000 radio_rx_freq 400300000 radio_rx_path 7
%si sleep 4
%si node 25
%si set sample_for 10
print("Started sampling at offset: ", start_offset)

In [ ]:
client.seek_stream('raw_stream', start_offset)
raw_data = client.read_stream('raw_stream', 2000000).view(np.complex64)
plt.psd(raw_data.view(np.complex64), NFFT=4096, Fs=200000);

In [ ]:
plt.plot(np.abs(raw_data.view(np.complex64)));
#plt.plot(np.real(raw_data.view(np.complex64))[:1000]);
#plt.plot(np.imag(raw_data.view(np.complex64))[:1000]);